In [63]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd
import numpy as np
import json
from ssl import create_default_context
from datetime import datetime
from datetime import timedelta
import requests
import math
#from getpass import getpass
import parametros


## Conectando a ElasticSearch

In [64]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=0, retry_on_timeout=True
)

### Fechas para el proceso

In [65]:
now = datetime.now()
new_date = now - timedelta(days=6)
format = "%Y%m%d"
format_ES = "%Y.%m.%d"
fecha_url = str(new_date.strftime(format))
fecha_hoy = str(now.strftime(format_ES))

### Función para formato JSON ES

In [66]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

## Leyendo la APi cambium-deviceclients

In [68]:
# Definimos la cabecera y el diccionario con los datos

datos_api = pd.DataFrame(columns=["mac","ip","manufacturer","name","user","radio","client_type"
                                  ,"total_quota","total_quota_balance","upload_quota","download_quota"
                                  ,"upload_quota_balance","download_quota_balance","ap_mac"
                                  ,"managed_account"])
j=0
for k in range(0,len(parametros.url_cambium)):
    token_aux = 'Bearer ' + parametros.cambium_token_aux[k]
    cabecera1 = { 'Content-Type': 'application/json', 'Authorization' : token_aux, 'accept' : '*/*' } 
    url = parametros.url_cambium[k] + "devices/clients/"
    r = requests.get(url, headers = cabecera1, verify=False)
    if r.status_code == 200:
        res = json.loads(r.text)
        dato_param = res['paging']
        #se calcula los ciclos de la consulta paginada
        #ciclos = 1  #int(math.ceil(dato_param['total']/100))
        ciclos = int(math.ceil(dato_param['total']/100))
        i = 0
        while i < ciclos:
            offset = str(i*100)
            i+=1
            #url2 = parametros.cambium_d_c_url + '?offset=' + offset
            url2 = url + '?offset=' + offset
            r = requests.get(url2, headers = cabecera1, verify=False)
            if r.status_code == 200:
                #print("respuesta 200: ",r.text)
                res = json.loads(r.text)
                #datos_api = datos_api.append(res['data'], ignore_index=True)
                for x in range(0,len(res['data'])):
                    #print(res['data'])
                    try:
                        mac = res['data'][x]['mac']
                    except KeyError:
                        mac = ''
                    try:
                        ip = res['data'][x]['ip']
                    except KeyError:
                        ip = ''
                    try:
                        manufacturer = res['data'][x]['manufacturer']
                    except KeyError:
                        manufacturer = ''
                    try:
                        name = res['data'][x]['name']
                    except KeyError:
                        name = ''
                    try:
                        user = res['data'][x]['user']
                    except KeyError:
                        user = ''
                    try:
                        radio = res['data'][x]['radio']
                    except KeyError:
                        radio = ''
                    try:
                        client_type = res['data'][x]['client_type']
                    except KeyError:
                        client_type = ''
                    try:
                        total_quota = res['data'][x]['total_quota']
                    except KeyError:
                        total_quota = ''
                    try:
                        total_quota_balance = res['data'][x]['total_quota_balance']
                    except KeyError:
                        total_quota_balance = ''
                    try:
                        upload_quota = res['data'][x]['upload_quota']
                    except KeyError:
                        upload_quota = ''
                    try:
                        download_quota = res['data'][x]['download_quota']
                    except KeyError:
                        download_quota = ''
                    try:
                        upload_quota_balance = res['data'][x]['upload_quota_balance']
                    except KeyError:
                        upload_quota_balance = ''
                    try:
                        download_quota_balance = res['data'][x]['download_quota_balance']
                    except KeyError:
                        download_quota_balance = ''
                    try:
                        ap_mac = res['data'][x]['ap_mac']
                    except KeyError:
                        ap_mac = ''
                    try:
                        managed_account = res['data'][x]['managed_account']
                    except KeyError:
                        managed_account = ''
                    try:    
                        datos_api.loc[j] = [mac,ip,manufacturer,name,user
                                         ,radio,client_type,total_quota,total_quota_balance,upload_quota
                                         ,download_quota,upload_quota_balance
                                         ,download_quota_balance,ap_mac,managed_account]
                    except:
                        pass
                    j+=1 # es el indice
                    
                print(url)
             
                print(len(datos_api.index))
            else:
                #break
                print("Fecha:",now,"- Error bucle interno: ",r.status_code)
        datos_api.dropna(how='all',inplace=True)
    else:
        print("Fecha:",now,"- Error bucle externo:",r.status_code)
        #exit()

/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.252'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.252'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://100.123.26.252/api/v2/devices/clients/
100


/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.252'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://100.123.26.252/api/v2/devices/clients/
200


/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.252'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


...

https://100.123.26.224/api/v2/devices/clients/
19839


/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.224'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://100.123.26.224/api/v2/devices/clients/
19857


/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.224'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://100.123.26.224/api/v2/devices/clients/
19857


In [69]:
datos_api

,mac,ip,manufacturer,name,user,radio,client_type,total_quota,total_quota_balance,upload_quota,download_quota,upload_quota_balance,download_quota_balance,ap_mac,managed_account
0,F6:73:4E:E2:64:4E,10.127.202.97,[Local MAC],Galaxy-A10-de-Shaky,F6-73-4E-E2-64-4E,"{'band': '2.4GHz', 'id': 1, 'rssi': -87, 'rx_b...",Guest Client,NA,NA,0,0,0,0,BC:E6:7C:4E:3D:AD,
1,8C:D9:D6:34:40:5C,0.0.0.0,8C-D9-D6,NA,,"{'band': '5GHz', 'id': 2, 'rssi': -91, 'rx_byt...",Guest Client,NA,NA,0,0,0,0,BC:E6:7C:4F:07:90,
2,60:D8:19:49:37:FC,10.23.143.73,Hon Hai Precision Ind Ltd,DESKTOP-FATIBJV,,"{'band': '2.4GHz', 'id': 1, 'rssi': -88, 'rx_b...",Client,,,,,,,BC:E6:7C:EC:AA:C0,
3,B4:1C:30:C2:1C:2B,10.123.143.95,ZTE Corporation,ZTE-Blade-L130,B4-1C-30-C2-1C-2B,"{'band': '2.4GHz', 'id': 1, 'rssi': -76, 'rx_b...",Guest Client,NA,NA,0,0,0,0,BC:E6:7C:5B:AF:F5,
4,FC:42:03:A8:96:45,10.123.143.177,Samsung Electronics Ltd,NA,FC-42-03-A8-96-45,"{'band': '2.4GHz', 'id': 1, 'rssi': -87, 'rx_b...",Guest Client,NA,NA,0,0,0,0,BC:E6:7C:5B:7C:0C,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19852,D4:C9:4B:20:71:4C,10.111.253.207,"Motorola Mobility LLC, a Lenovo Company",NA,,"{'band': '2.4GHz', 'id': 1, 'rssi': -62, 'rx_b...",Guest Client,NA,NA,0,0,0,0,BC:A9:93:62:6C:7C,
19853,CA:E6:68:A9:B1:DB,10.111.253.210,[Local MAC],NA,,"{'band': '2.4GHz', 'id': 1, 'rssi': -59, 'rx_b...",Guest Client,NA,NA,0,0,0,0,BC:A9:93:62:6C:7C,
19854,00:12:36:D3:D5:5D,10.111.253.209,ConSentry Networks,Galaxy-J7-Sky-Pro,,"{'band': '2.4GHz', 'id': 1, 'rssi': -63, 'rx_b...",Guest Client,NA,NA,0,0,0,0,BC:A9:93:62:6C:7C,
19855,08:AA:55:E2:36:33,10.101.64.210,08-AA-55,NA,,"{'band': '2.4GHz', 'id': 1, 'rssi': -61, 'rx_b...",Guest Client,NA,NA,0,0,0,0,BC:A9:93:0C:79:5F,


### Se eliminan duplicados por mac

In [70]:
datos_api.drop_duplicates(subset=['mac'],inplace=True)
#datos_api.drop_duplicates(subset=['ap_mac','mac'],inplace=True)

In [77]:
datos_api

,mac,ip,manufacturer,name,user,client_type,total_quota,total_quota_balance,upload_quota,download_quota,...,download_quota_balance,ap_mac,managed_account,radio.band,radio.rssi,radio.rx_bytes,radio.snr,radio.ssid,radio.tx_bytes,@timestamp
0,F6:73:4E:E2:64:4E,10.127.202.97,[Local MAC],Galaxy-A10-de-Shaky,F6-73-4E-E2-64-4E,Guest Client,NA,NA,,,...,,BC:E6:7C:4E:3D:AD,,2.4GHz,-87,436006,8,MINTIC_CONECTA,13226886,2021-11-04T12:13:02.681967
1,8C:D9:D6:34:40:5C,0.0.0.0,8C-D9-D6,NA,,Guest Client,NA,NA,,,...,,BC:E6:7C:4F:07:90,,5GHz,-91,1314,4,MINTIC_CONECTA,0,2021-11-04T12:13:02.681967
2,60:D8:19:49:37:FC,10.23.143.73,Hon Hai Precision Ind Ltd,DESKTOP-FATIBJV,,Client,,,,,...,,BC:E6:7C:EC:AA:C0,,2.4GHz,-88,995043,7,MINTIC,786187,2021-11-04T12:13:02.681967
3,B4:1C:30:C2:1C:2B,10.123.143.95,ZTE Corporation,ZTE-Blade-L130,B4-1C-30-C2-1C-2B,Guest Client,NA,NA,,,...,,BC:E6:7C:5B:AF:F5,,2.4GHz,-76,558381,19,MINTIC_CONECTA,5572599,2021-11-04T12:13:02.681967
4,FC:42:03:A8:96:45,10.123.143.177,Samsung Electronics Ltd,NA,FC-42-03-A8-96-45,Guest Client,NA,NA,,,...,,BC:E6:7C:5B:7C:0C,,2.4GHz,-87,16273,8,MINTIC_CONECTA,28088,2021-11-04T12:13:02.681967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19852,D4:C9:4B:20:71:4C,10.111.253.207,"Motorola Mobility LLC, a Lenovo Company",NA,,Guest Client,NA,NA,,,...,,BC:A9:93:62:6C:7C,,2.4GHz,-62,119055,33,MINTIC_CONECTA,4454,2021-11-04T12:13:02.681967
19853,CA:E6:68:A9:B1:DB,10.111.253.210,[Local MAC],NA,,Guest Client,NA,NA,,,...,,BC:A9:93:62:6C:7C,,2.4GHz,-59,135069,36,MINTIC_CONECTA,5542,2021-11-04T12:13:02.681967
19854,00:12:36:D3:D5:5D,10.111.253.209,ConSentry Networks,Galaxy-J7-Sky-Pro,,Guest Client,NA,NA,,,...,,BC:A9:93:62:6C:7C,,2.4GHz,-63,77331,32,MINTIC_CONECTA,3688,2021-11-04T12:13:02.681967
19855,08:AA:55:E2:36:33,10.101.64.210,08-AA-55,NA,,Guest Client,NA,NA,,,...,,BC:A9:93:0C:79:5F,,2.4GHz,-61,16479,34,MINTIC_CONECTA,400,2021-11-04T12:13:02.681967


### Se descompone el diccionario radio

In [72]:
datos_api['radio.band'] = datos_api.loc[:, 'radio'].apply(lambda x: x['band'])
datos_api['radio.rssi'] = datos_api.loc[:, 'radio'].apply(lambda x: x['rssi'])
datos_api['radio.rx_bytes'] = datos_api.loc[:, 'radio'].apply(lambda x: x['rx_bytes'])
datos_api['radio.snr'] = datos_api.loc[:, 'radio'].apply(lambda x: x['snr'])
datos_api['radio.ssid'] = datos_api.loc[:, 'radio'].apply(lambda x: x['ssid'])
datos_api['radio.tx_bytes'] = datos_api.loc[:, 'radio'].apply(lambda x: x['tx_bytes'])
datos_api.drop(columns=['radio'],inplace=True)

In [73]:
datos_api['upload_quota'] = datos_api['upload_quota'].str.replace('NA','')
datos_api['upload_quota_balance'] = datos_api['upload_quota_balance'].str.replace('NA','')
datos_api['download_quota'] = datos_api['download_quota'].str.replace('NA','')
datos_api['download_quota_balance'] = datos_api['download_quota_balance'].str.replace('NA','')

In [74]:
datos_api.fillna('', inplace=True)

### Definiendo indice con fecha e insertando en ES

In [75]:
indice = 'cambium_clients_pruebainges'

In [76]:
use_these_keys = ['mac', 'ip', 'manufacturer', 'name', 'user', 'client_type','total_quota'
                  , 'total_quota_balance', 'upload_quota', 'download_quota','upload_quota_balance'
                  , 'download_quota_balance', 'ap_mac','managed_account', 'radio.band', 'radio.rssi'
                  , 'radio.rx_bytes','radio.snr', 'radio.ssid', 'radio.tx_bytes','@timestamp']
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

timestamp = datetime.now()
datos_api['@timestamp'] = timestamp.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{document['mac']}",
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(datos_api))
print("total documentos insertados: ",salida[0])

total documentos insertados:  12390


# Inserta con con fecha_control

In [48]:
use_these_keys = ['mac', 'ip', 'manufacturer', 'name', 'user'
                  , 'client_type', 'ap_mac','managed_account'
                  , 'radio.band', 'radio.rx_bytes', 'radio.tx_bytes'
                  ,'fecha_control','@timestamp']
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }
datos_api['fecha_control'] = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:-3]+':00'
datos_api['@timestamp'] = now.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": 'alltest-'+indice, 
                "_id": f"{str(document['mac'])+'-'+str(document['ap_mac'])+'-'+str(document['fecha_control'])}",
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(datos_api))
print("fecha:",str(now.strftime('%Y-%m-%d %H:%M:%S')),"total documentos con fecha control: ",salida[0])

fecha: 2021-11-02 21:46:15 total documentos con fecha control:  200


### Se calculan datos agrupados

Se calcula para cada ap_mac:
* conteo por usuarios.marcaTerminal
* conteo por usuarios.detallesTecnologiasTerminales

Los resultados se almacenan dentro del mismo indice. Para garantizar la no duplicidad de registro por mac, se pone el prefito 'totales' al _id

In [56]:
total_docs = 10000
response = es.search(
    index= 'cambium_clients_pruebainges',
    body={
            "_source": ["mac","ap_mac","radio.band","radio.rx_bytes","radio.tx_bytes","manufacturer"]
    },
    size=total_docs
)
elastic_docs = response["hits"]["hits"]

fields = {}
for num, doc in enumerate(elastic_docs):
    source_data = doc["_source"]
    for key, val in source_data.items():
        try:
            fields[key] = np.append(fields[key], val)
        except KeyError:
            fields[key] = np.array([val])

datos_dev_clients = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
datos_dev_clients.drop_duplicates(subset=['ap_mac','mac'],inplace=True)

In [52]:
datos_dev_clients

,ap_mac,mac,radio.band,radio.rx_bytes,radio.tx_bytes,manufacturer
0,BC:E6:7C:5B:7D:4B,AC:37:43:03:60:FF,2.4GHz,1496,1340,HTC Corporation
1,BC:E6:7C:5B:7D:04,AC:E3:42:10:0B:98,2.4GHz,215131,293626,Huawei Technologies Ltd
2,58:C1:7A:0D:C8:63,58:63:56:92:D9:9D,2.4GHz,1162861,25727330,FN-Link Technology Limited
3,BC:E6:7C:ED:F2:BA,28:E3:47:A3:C2:12,2.4GHz,630437,557838,Liteon Technology Corporation
4,BC:E6:7C:4F:07:90,34:1C:F0:25:C6:73,2.4GHz,121673,71066,34-1C-F0
...,...,...,...,...,...,...
7103,BC:E6:7C:5D:20:6A,00:E9:3A:D2:82:63,5GHz,37083,0,00-E9-3A
7104,BC:E6:7C:5D:20:6A,B2:AF:02:8B:A6:C0,2.4GHz,4678,0,[Local MAC]
7105,BC:E6:7C:5D:20:6A,20:34:FB:CC:64:43,5GHz,2654,0,Xiaomi Communications Ltd
7106,BC:E6:7C:5D:88:AF,E8:03:9A:C2:88:56,2.4GHz,85308537,76646086,Samsung Electronics Ltd


In [50]:
fabricantes = datos_dev_clients[['ap_mac','manufacturer','mac']].groupby(['ap_mac','manufacturer']).agg(['count']).reset_index()
fabricantes.columns = fabricantes.columns.droplevel(1)
fabricantes= fabricantes.rename(columns={'ap_mac': 'usuarios.macRed',
                                         'manufacturer' : 'usuarios.marcaTerminal',
                                         'mac' : 'usuarios.total.marcaTerminal'})

In [51]:
use_these_keys = ['usuarios.macRed', 'usuarios.total.marcaTerminal', 'usuarios.marcaTerminal','@timestamp']

fabricantes['@timestamp'] = now.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'Totales-'+document['usuarios.macRed'] + '-' +document['usuarios.marcaTerminal']}",
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(fabricantes))
print("total totales fabricantes insertados: ",salida[0])

total totales fabricantes insertados:  5870


In [57]:
datos_dev_clients.fillna({'radio.rx_bytes':0,'radio.tx_bytes':0},inplace=True)
datos_dev_clients['usuarios.usoServicioInternetSitio'] = datos_dev_clients['radio.rx_bytes'] + datos_dev_clients['radio.tx_bytes']
radios = datos_dev_clients[['ap_mac','radio.band','usuarios.usoServicioInternetSitio']].groupby(['ap_mac','radio.band']).agg(['sum']).reset_index()
radios.columns = radios.columns.droplevel(1)
radios= radios.rename(columns={'ap_mac': 'usuarios.macRed',
                               'radio.band' : 'usuarios.detallesTecnologiasTerminales'})

In [58]:
use_these_keys = ['usuarios.macRed', 'usuarios.detallesTecnologiasTerminales', 'usuarios.usoServicioInternetSitio','@timestamp']

radios['@timestamp'] = now.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'Totales-'+document['usuarios.macRed']}",
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(radios))
print("total totales Uso servicio insertados: ",salida[0])
print("Fin")

total totales Uso servicio insertados:  330
Fin
